In [30]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [31]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [32]:
! nvidia-smi

Sat Jul  2 14:33:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 27%   35C    P8    20W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 26%   

# MongoDB

In [33]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn
    
    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de

    db_rm_us_2010 = db.rm_patent_us_2010
    db_rm_cn_2010 = db.rm_patent_cn_2010
    db_rm_de_2010 = db.rm_patent_de_2010
    
except errors.ConnectionFailure as err:
    print(err)

In [34]:
db_rm_us_2010

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2010')

# US

In [35]:
df = pd.DataFrame(list(db_rm_us.find()))

In [36]:
df["countrystatus"][0]

['US',
 'EP',
 'JP',
 'KR',
 'CN',
 'AU',
 'CA',
 'DK',
 'MX',
 'MY',
 'RU',
 'SG',
 'WO',
 'ZA']

In [37]:
df.head(2)

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,62009a807b73c16636b31084,US20190027414A1,"[Senthil Kumar Ramadas, Saravanan Shanmugavel]","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...",20180924,20190124,N/A,"[B32B7/02, H01L23/29, B82Y30/00, C23C28/00, C2...","Disclosed is an encapsulation barrier stack, c...",Encapsulationbarrierstack,The present application is a divisional applic...,[1.Amethodofmanufacturinganencapsulationbarrie...,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...","[{'pto': 'US', 'patentNumber': '20100089636', ...","[{'pto': 'US', 'patentNumber': '560778', 'kind...","[{'pto': 'US', 'patentNumber': '10854853', 'ki...","[{'pto': 'US', 'patentNumber': '20150314941', ...","[{'publication': 'US20190027414A1', 'time': '2...",US201161550764P
1,62009b557b73c16636b31088,US20140310243A1,"[Steven James McGee, Tammy Lee McGee]",[],[Mr. Steven James McGee],20120813,20141016,N/A,"[G06F17/30575, G06F16/27, G06Q10/101, G06Q10/1...","Systemic, adaptive procedural template compris...",Heartbeaconcycle,Related applications by Applicant Steven J. Mc...,"[1.Asystemic,adaptiveproceduraltemplatemethodu...",[US],"[{'pto': 'US', 'patentNumber': '20060217138', ...",None,"[{'pto': 'US', 'patentNumber': '20130086129', ...",None,"[{'publication': 'US20140310243A1', 'time': '2...","US12/856,715"


In [38]:
df_us = df.loc[:,["patentNumber","countrystatus"]]
df_us.head(3)

,patentNumber,countrystatus
0,US20190027414A1,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S..."
1,US20140310243A1,[US]
2,US8965375B2,"[US, EP, WO]"


In [39]:
df_us["count"] = df_us.countrystatus.apply(lambda x: len(x))

In [40]:
df_us.head(3)

,patentNumber,countrystatus,count
0,US20190027414A1,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...",14
1,US20140310243A1,[US],1
2,US8965375B2,"[US, EP, WO]",3


In [41]:
df_us_family_size = df_us.groupby(["count"],as_index=False).size()
# df_us_family_size

In [42]:
df_us_family_size["ratio"] = df_us_family_size.apply(lambda x: round(x["size"]/sum(df_us_family_size["size"])*100,2), axis=1)
df_us_family_size.sort_values(by=['ratio'],ascending=False)

,count,size,ratio
1,1,14216,33.89
2,2,9562,22.79
3,3,6244,14.88
4,4,5906,14.08
5,5,2692,6.42
6,6,1451,3.46
7,7,661,1.58
8,8,406,0.97
9,9,348,0.83
0,0,166,0.40


# DE

In [43]:
df = pd.DataFrame(list(db_rm_de.find()))

In [44]:
df["countrystatus"][0]

['US', 'JP', 'CN', 'CA', 'DE', 'GB', 'IN', 'WO']

In [45]:
df.head(2)

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fc144a5eb7331c4068bc4a,DE112012005419B4,"[Chi Liu, Qi Yu, Zhentan Feng, Bo Yang, Zhanwe...","[GlobalFoundries Inc, GlobalFoundries US Inc]",[GlobalFoundries Inc],20121031,20190718,N/A,"[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,SystemundVerfahrenfüreineeffizientedienstinsta...,Technisches GebietDie vorliegende Erfindung be...,[DienstinstanzenorientiertesEnergieverwaltungs...,"[US, JP, CN, CA, DE, GB, IN, WO]","[{'pto': 'CN', 'patentNumber': '10214352', 'ki...","[{'pto': 'JP', 'patentNumber': '200227860', 'k...",None,"[{'pto': 'CN', 'patentNumber': '10341505', 'ki...","[{'publication': 'DE112012005419B4', 'time': '...",CN201110456813.8A
1,61fc14c95eb7331c4068bc4c,DE102012022880A1,"[René Peter Scheurich, Junyou Pan, Amir Hossai...",[Merck Patent GmbH],[Merck Patent GmbH],20121123,20130627,N/A,"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,ElektronischeVorrichtungenenthaltendorganische...,Die vorliegende Erfindung betrifft eine elektr...,[ElektronischeVorrichtungenthaltendmindestense...,[DE],"[{'pto': 'US', 'patentNumber': '453950', 'kind...",None,"[{'pto': 'CN', 'patentNumber': '10348333', 'ki...",None,"[{'publication': 'EP3131901B1', 'time': '2018-...",EP11010151.6


In [46]:
df_de = df.loc[:,["patentNumber","countrystatus"]]
df_de.head(3)

,patentNumber,countrystatus
0,DE112012005419B4,"[US, JP, CN, CA, DE, GB, IN, WO]"
1,DE102012022880A1,[DE]
2,DE102012218966B4,"[US, DE, GB]"


In [47]:
df_de["count"] = df_de.countrystatus.apply(lambda x: len(x))

In [48]:
df_de.head(3)

,patentNumber,countrystatus,count
0,DE112012005419B4,"[US, JP, CN, CA, DE, GB, IN, WO]",8
1,DE102012022880A1,[DE],1
2,DE102012218966B4,"[US, DE, GB]",3


In [49]:
df_de_family_size = df_de.groupby(["count"],as_index=False).size()
# df_de_family_size

In [50]:
df_de_family_size["ratio"] = df_de_family_size.apply(lambda x: round(x["size"]/sum(df_de_family_size["size"])*100,2), axis=1)
df_de_family_size.sort_values(by=['ratio'],ascending=False)

,count,size,ratio
0,1,343,29.88
1,2,281,24.48
2,3,197,17.16
3,4,159,13.85
4,5,100,8.71
5,6,31,2.70
6,7,20,1.74
7,8,6,0.52
9,10,4,0.35
8,9,3,0.26


# CN

In [51]:
df = pd.DataFrame(list(db_rm_cn.find()))

In [52]:
df["countrystatus"][0]

['US',
 'EP',
 'JP',
 'KR',
 'CN',
 'AP',
 'AU',
 'BR',
 'CA',
 'CL',
 'CO',
 'CR',
 'CU',
 'EA',
 'EC',
 'GT',
 'HK',
 'IL',
 'MA',
 'MX',
 'PE',
 'SG',
 'TN',
 'TW',
 'WO',
 'ZA']

In [53]:
df.head(2)

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fd25e1f4d57576a5021669,CN103492389B,"[M·阿尔姆施泰特尔, M·托尔曼, A·特雷姆尔, N·特劳贝]",[Origenis GmbH],[Origenis GmbH],20120423,20160914,N/A,"[C07D487/04, A61K31/519, A61P1/00, A61P11/00, ...",本发明涉及式(I)的新化合物，其能够抑制一种或多种激酶，尤其抑制SYK(脾酪氨酸激酶)、LR...,"用作激酶抑制剂的吡唑并[4,3-d]嘧啶",技术领域[0001 ]本发明涉及能够抑制一种或多种激酶，尤其SYK(脾酪氨酸激酶）、LRRK...,[1.式(I)的化合物或其药学上可接受的盐其中A为NH;Ri为任选地被取代的杂烷基、C6-1...,"[US, EP, JP, KR, CN, AP, AU, BR, CA, CL, CO, C...","[{'pto': 'WO', 'patentNumber': '1998029413', '...","[{'pto': 'US', 'patentNumber': '446986', 'kind...",None,"[{'pto': 'KR', 'patentNumber': '2014002097', '...","[{'publication': 'CN103492389B', 'time': '2016...",US201161517582P
1,61fd25e5f4d57576a502166b,CN102393961A,"[王滨海, 张晶晶, 王骞, 刘俍, 王万国]",[State Grid Intelligent Technology Co Ltd],[Electric Power Research Institute of State Gr...,20110629,20120328,N/A,None,本发明公开了一种在复杂自然环境背景下，实现了基于计算机视觉的飞行器输电线路实时检测跟踪方法。...,基于计算机视觉的飞行器输电线路实时检测跟踪方法,基于计算机视觉的飞行器输电线路实时检测跟踪方法技术领域[0001]\t本发明涉及一种图像处理...,[1.一种基于计算机视觉的飞行器输电线路实时检测跟踪方法，其特征是，首先使用抗噪声的梯度图像...,[CN],None,None,"[{'pto': 'CN', 'patentNumber': '10314993', 'ki...",None,"[{'publication': 'CN102393961A', 'time': '2012...",CN201110179733.2A


In [54]:
df_cn = df.loc[:,["patentNumber","countrystatus"]]
df_cn.head(3)

,patentNumber,countrystatus
0,CN103492389B,"[US, EP, JP, KR, CN, AP, AU, BR, CA, CL, CO, C..."
1,CN102393961A,[CN]
2,CN103155174B,"[US, EP, JP, KR, CN, AU, CA, WO]"


In [55]:
df_cn["count"] = df_cn.countrystatus.apply(lambda x: len(x))

In [56]:
df_cn.head(3)

,patentNumber,countrystatus,count
0,CN103492389B,"[US, EP, JP, KR, CN, AP, AU, BR, CA, CL, CO, C...",26
1,CN102393961A,[CN],1
2,CN103155174B,"[US, EP, JP, KR, CN, AU, CA, WO]",8


In [57]:
df_cn_family_size = df_cn.groupby(["count"],as_index=False).size()
# df_cn_family_size

In [58]:
df_cn_family_size["ratio"] = df_cn_family_size.apply(lambda x: round(x["size"]/sum(df_cn_family_size["size"])*100,2), axis=1)
df_cn_family_size.sort_values(by=['ratio'],ascending=False)

,count,size,ratio
1,1,27915,76.03
2,2,2679,7.30
4,4,1797,4.89
3,3,1567,4.27
5,5,1104,3.01
6,6,591,1.61
7,7,307,0.84
8,8,216,0.59
9,9,182,0.50
0,0,159,0.43
